In [1]:
import pandas as pd
import io
import requests
import psycopg2
from sqlalchemy import create_engine

In [2]:
GITHUB_BASE_URL = "https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/"

In [3]:
def fetch_csv(technology):
    """Fetches and merges only available CSV files for a given technology."""
    df_list = []
    available_days = []
    
    for day in range(1, 32):  # Checking for all possible days (1 to 31)
        url = f"{GITHUB_BASE_URL}{technology}day{day}.csv"
        try:
            response = requests.get(url)
            if response.status_code == 200:
                available_days.append(day)  # Store available days
                print(f"Fetching: {url}")
                
                # Read CSV with correct delimiter
                csv_data = io.StringIO(response.text)
                df = pd.read_csv(csv_data, sep=';')  # ✅ Fix: Explicitly set `sep=';'`
                
                # Trim whitespace from column names
                df.columns = df.columns.str.strip()
                
                df_list.append(df)  
                
        except requests.RequestException as e:
            print(f"Error fetching {url}: {e}")  # Debugging print
    
    print(f"Available days for {technology}: {available_days}")
    return pd.concat(df_list, ignore_index=True) if df_list else pd.DataFrame()


In [4]:
# Load CSV Data from GitHub
gsm_df = fetch_csv("gsm")
umts_df = fetch_csv("umts")
lte_df = fetch_csv("lte")
site_df = fetch_csv("site")


print(f"GSM DataFrame Rows: {len(gsm_df)}")
print(f"UMTS DataFrame Rows: {len(umts_df)}")
print(f"LTE DataFrame Rows: {len(lte_df)}")
print(f"Site DataFrame Rows: {len(site_df)}")


Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/gsmday24.csv
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/gsmday25.csv
Available days for gsm: [24, 25]
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/umtsday24.csv
Available days for umts: [24]
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/lteday24.csv
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/lteday25.csv
Available days for lte: [24, 25]
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/siteday24.csv
Fetching: https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/siteday25.csv
Available days for site: [24, 25]
GSM DataFrame Rows: 4
UMTS DataFrame Rows: 2
LTE DataFrame Rows: 8
Site DataFrame Rows: 3


In [5]:
# Assign Technology Labels
gsm_df["technology"] = "2g" if not gsm_df.empty else None
umts_df["technology"] = "3g" if not umts_df.empty else None
lte_df["technology"] = "4g" if not lte_df.empty else None

In [6]:
# Merge All Cells
total_cells_df = pd.concat([gsm_df, umts_df, lte_df], ignore_index=True)
total_cells_df,site_df

(    year  month  day cell_identity  frequency_band  site_id technology
 0   2018     10   24             C            1800        3         2g
 1   2018     10   24             D            1800        3         2g
 2   2018     10   25             A             900        1         2g
 3   2018     10   25             B            1800        1         2g
 4   2018     10   24             E            2100        3         3g
 5   2018     10   24             F            2100        3         3g
 6   2018     10   24             I             700        3         4g
 7   2018     10   24             J            1800        3         4g
 8   2018     10   24             K             800        3         4g
 9   2018     10   24             L            2600        3         4g
 10  2018     10   24             M            1800        3         4g
 11  2018     10   25             G             900        1         4g
 12  2018     10   25             H            2100        1    

In [7]:
# Join with Site Data
merged_df = total_cells_df.merge(site_df, on="site_id", how="inner")
merged_df

,year_x,month_x,day_x,cell_identity,frequency_band,site_id,technology,year_y,month_y,day_y
0,2018,10,24,C,1800,3,2g,2018,10,24
1,2018,10,24,D,1800,3,2g,2018,10,24
2,2018,10,25,A,900,1,2g,2018,10,25
3,2018,10,25,B,1800,1,2g,2018,10,25
4,2018,10,24,E,2100,3,3g,2018,10,24
5,2018,10,24,F,2100,3,3g,2018,10,24
6,2018,10,24,I,700,3,4g,2018,10,24
7,2018,10,24,J,1800,3,4g,2018,10,24
8,2018,10,24,K,800,3,4g,2018,10,24
9,2018,10,24,L,2600,3,4g,2018,10,24


In [8]:
# Count Cells Per Technology Per Site
cell_count = merged_df.groupby("site_id")["technology"].value_counts().unstack(fill_value=0)
cell_count.columns = [f"site_{col}_cnt" for col in cell_count.columns]
cell_count

,site_2g_cnt,site_3g_cnt,site_4g_cnt
site_id,,,
1,2,0,2
3,2,2,5


In [9]:
# Calculate Frequency Bands Per Site
band_df = merged_df.groupby(["site_id", "technology", "frequency_band"]).size().unstack(fill_value=0)
band_df.columns = [f"frequency_band_{col}_by_site" for col in band_df.columns]
band_df

frequency_band_700_by_site  frequency_band_800_by_site  \
site_id technology                                                           
1       2g                                   0                           0   
        4g                                   0                           0   
3       2g                                   0                           0   
        3g                                   0                           0   
        4g                                   1                           1   

                    frequency_band_900_by_site  frequency_band_1800_by_site  \
site_id technology                                                            
1       2g                                   1                            1   
        4g                                   1                            0   
3       2g                                   0                            2   
        3g                                   0                            0   
        4g                                   0                            2   

                    frequency_band_2100_by_site  frequency_band_2600_by_site  
site_id technology                                                            
1       2g                                    0                            0  
        4g                                    1                            0  
3       2g                                    0                            0  
        3g                                    2                            0  
        4g                                    0                            1

In [10]:
# Combine all results
final_df = cell_count.join(band_df, how="left")
final_df

site_2g_cnt  site_3g_cnt  site_4g_cnt  \
site_id technology                                          
1       2g                    2            0            2   
        4g                    2            0            2   
3       2g                    2            2            5   
        3g                    2            2            5   
        4g                    2            2            5   

                    frequency_band_700_by_site  frequency_band_800_by_site  \
site_id technology                                                           
1       2g                                   0                           0   
        4g                                   0                           0   
3       2g                                   0                           0   
        3g                                   0                           0   
        4g                                   1                           1   

                    frequency_band_900_by_site  frequency_band_1800_by_site  \
site_id technology                                                            
1       2g                                   1                            1   
        4g                                   1                            0   
3       2g                                   0                            2   
        3g                                   0                            0   
        4g                                   0                            2   

                    frequency_band_2100_by_site  frequency_band_2600_by_site  
site_id technology                                                            
1       2g                                    0                            0  
        4g                                    1                            0  
3       2g                                    0                            0  
        3g                                    2                            0  
        4g                                    0                            1

In [11]:
# Saving Results to PostgreSQL
from sqlalchemy import create_engine
import pandas as pd

#PostgreSQL connection details
db_username = 'postgres'
db_password = 'gayathri98!'
db_host = 'localhost'  
db_port = '5432'        
db_name = 'TELECOM'

# Creating the connection string
connection_string = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'

# Creating a SQLAlchemy engine
engine = create_engine(connection_string)

# Loading data into a DataFrame
df = final_df

# Load data into PostgreSQL
table_name = 'site_status'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

5

In [12]:
print("Processing complete! Data stored in PostgreSQL database.")

Processing complete! Data stored in PostgreSQL database.


In [13]:
import requests

response = requests.get("https://raw.githubusercontent.com/GayathriBS25/TelecomSiteAnalysis/main/data/gsmday24.csv")
print(response.status_code)


200
